<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
from cryptography.fernet import Fernet
import pickle
import warnings
from bs4 import BeautifulSoup
import requests
warnings.simplefilter(action='ignore', category=Warning)

In [2]:
with open('secrets/secret.pkl', 'rb') as file:
    tmp = pickle.load(file)

    cipher = Fernet(tmp['salt'])

pass_data = {
    'dbname': cipher.decrypt(tmp['dbname']).decode('utf-8'),
    'user': cipher.decrypt(tmp['user']).decode('utf-8'),
    'password': cipher.decrypt(tmp['password']).decode('utf-8'),
    'host': cipher.decrypt(tmp['host']).decode('utf-8'),
    'port': int(cipher.decrypt(tmp['port']).decode('utf-8')),
}

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [3]:
# текст запроса
query_3_1 = f'''SELECT COUNT(id)
                FROM vacancies
'''

In [4]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_3_1, conn)

df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [5]:
query_3_2 = f'''SELECT COUNT(id)
                FROM employers
'''

In [6]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_3_2, conn)

df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [7]:
query_3_3 = f'''SELECT COUNT(id)
                FROM areas
'''

In [8]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_3_3, conn)

df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [9]:
query_3_4 = f'''SELECT COUNT(id)
                FROM industries
'''

In [10]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_3_4, conn)

df

,count
0,294


***

## Выводы:
* Представленное количество вакансий и количество работодателей различаются почти в два раза. Что может говорить о том, что не все работодатели представили свои вакансии.
* Данные представляют обширную георгафию.
* Представлено большое количество индустрий.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [11]:
query_4_1 = f'''SELECT a.name as area,
                       COUNT(v.id) as cnt 
                FROM vacancies as v                     
                JOIN areas as a ON v.area_id = a.id
                GROUP BY a.name
                ORDER BY cnt DESC
                LIMIT 5
'''

In [12]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_4_1, conn)

df

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [13]:
query_4_2 = f'''SELECT COUNT(id)
                FROM vacancies
                WHERE salary_from IS NOT NULL OR
                    salary_to IS NOT NULL
'''

In [14]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_4_2, conn)

df

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [15]:
query_4_3 = f'''SELECT ROUND(AVG(salary_from)) as avg_from,
                       ROUND(AVG(salary_to)) as avg_to
                FROM vacancies
'''

In [16]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_4_3, conn)

df

,avg_from,avg_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [17]:
query_4_4 = f'''SELECT schedule,
                       employment,
                       COUNT(id) as cnt
                FROM vacancies
                GROUP BY schedule, employment
                ORDER BY cnt DESC
'''

In [18]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_4_4, conn)

df

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [19]:
query_4_5 = f'''SELECT experience,
                       COUNT(id) as cnt
                FROM vacancies
                GROUP BY experience
                ORDER BY cnt
'''

In [20]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_4_5, conn)

df

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## Выводы:
* Наибольшое количество вакансий представлено в крупных городах.
* Средняя зарплата варируется от 71 до 110 тыс. рублей.
* Наибольшее число вакансий представлено для людей с опытом от 1 до 3 лет на полную занятость.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [21]:
query_5_1 = f'''(SELECT e.name as area,
                    COUNT(v.id) as cnt 
                FROM vacancies as v                     
                JOIN employers as e ON v.employer_id = e.id
                GROUP BY e.name
                ORDER BY cnt DESC
                LIMIT 1)
                
                UNION
                
                (SELECT e.name as area,
                    COUNT(v.id) as cnt 
                FROM vacancies as v                     
                JOIN employers as e ON v.employer_id = e.id
                GROUP BY e.name
                ORDER BY cnt DESC
                LIMIT 1
                OFFSET 4)
                
                ORDER BY cnt DESC
'''

In [22]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_5_1, conn)

df

,area,cnt
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [23]:
query_5_2 = f'''SELECT a.name,
                       count(distinct e.id) count_employers,
                       count(distinct v.id) count_vacancies
                FROM public.areas a
                    LEFT JOIN public.employers e ON e.area = a.id
                    LEFT JOIN public.vacancies v ON v.area_id = a.id
                WHERE v.id IS NULL
                GROUP BY a.name
                ORDER BY count(distinct e.id) DESC

'''

In [24]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_5_2, conn)

df

,name,count_employers,count_vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
588,Кувандык,0,0
589,Куйбышев,0,0
590,Куйбышево (Ростовская область),0,0
591,Култаево,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [25]:
query_5_3 = f'''SELECT e.name,
                       COUNT(DISTINCT(v.area_id)) AS area_cnt
                FROM vacancies AS v
                    LEFT JOIN areas AS a ON a.id = v.area_id
                    LEFT JOIN employers AS e ON e.id = v.employer_id
                GROUP BY e.name
                ORDER BY area_cnt DESC
'''

In [26]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_5_3, conn)

df

,name,area_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [27]:
query_5_4 = f'''SELECT COUNT(e.name)
                FROM employers AS e
                    LEFT JOIN employers_industries AS ei ON e.id = ei.employer_id
                WHERE industry_id IS NULL
'''

In [28]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_5_4, conn)

df

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [29]:
query_5_5 = f'''SELECT e.name,
                       COUNT(DISTINCT(industry_id)) as ind_cnt
                FROM employers AS e
                    LEFT JOIN employers_industries AS ei ON e.id = ei.employer_id
                GROUP BY e.name
                HAVING COUNT(DISTINCT(industry_id))  = 4
                ORDER BY e.name
                LIMIT 1
                OFFSET 2
'''

In [30]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_5_5, conn)

df

,name,ind_cnt
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [31]:
query_5_5 = f'''SELECT COUNT(e.name)
                FROM employers_industries AS ei
                    JOIN employers AS e ON e.id = ei.employer_id
                    JOIN industries AS i ON i.id = ei.industry_id
                WHERE i.name LIKE('Разработка программного обеспечения')
'''

In [32]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_5_5, conn)

df

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [33]:
URL = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'

# Делаем запрос, получем страничку
url_link = requests.get(URL)
# Иницируем объект класса ВолшеногоМыла
bs = BeautifulSoup(url_link.text)
# Найдем все таблицы в документе
table = bs.find_all('table')

# Посмотрев данные, я знаю, что нужная таблица это вторая,
# парсим строки и заголовки столбцов
table_rows = table[1].find_all('tr')
heads = table[1].find_all('th')
col_names = [h.text for h in heads]
# Распарсим отдельные элементы строк
standing_rows = []
rank = 1
for tr in table_rows:
    cols = tr.find_all('td')
    if cols:
        row = [tr.text for tr in cols[1:6]]
        standing_rows.append([rank]+row)
        rank += 1
        
# Запихнем в датафрейм
df_millionar = pd.DataFrame(standing_rows, columns=col_names)

df_millionar['Город'] = df_millionar['Город'].apply(lambda x: x.strip().replace('[a]',''))

df_millionar

,№,Город,"Население, тыс. чел. (итоги переписи 2021 года)[5]",Прирост относительно переписи-2010,"Население, тыс. чел.(итоги переписи 2010 года)[6]"
0,1,Москва,13 010,"13,1%",11 504
1,2,Санкт-Петербург,5602,"14,8%",4880
2,3,Новосибирск,1634,"10,9%",1474
3,4,Екатеринбург,1544,"14,4%",1350
4,5,Казань,1309,"14,4%",1144
5,6,Нижний Новгород,1226,"0,2−2,0%",1251
6,7,Челябинск,1190,"5,3%",1130
7,8,Красноярск,1188,"21,6%",977
8,9,Самара,1173,"0,3%",1169
9,10,Уфа,1145,"7,8%",1062


***

In [34]:
milliona_list =  tuple(df_millionar['Город'])

query_5_5 = f'''WITH cte AS (SELECT a.name,
                       COUNT(v.id) as cnt
                FROM vacancies AS v
                    JOIN employers AS e ON e.id = v.employer_id
                    LEFT JOIN areas AS a ON a.id = v.area_id
                WHERE e.name LIKE('Яндекс') AND
                      a.name IN {milliona_list}
                GROUP BY a.name
                ORDER by cnt)
                
                SELECT name,
                       cnt   
                FROM cte
                
                UNION
                SELECT 'Total',
                       SUM(cnt)
                FROM cte
                
                ORDER BY cnt
'''

In [35]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_5_5, conn)

df

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


## Выводы:
* Самыми активными в поиске сотрудников выступают крупные ИТ-компании и технологические гиганты.
* Чуть более трети компаний не указывают свою сферу деятельности.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [36]:
query_6_1 = f'''SELECT COUNT(id) AS cnt
                FROM vacancies AS v
                WHERE v.name ILIKE('%data%') OR
                      v.name ILIKE('%данн%')
'''

In [37]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_6_1, conn)

df

,cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [38]:
query_6_2 = f'''SELECT COUNT(id) AS cnt
                FROM vacancies
                WHERE (
                        (
                            name ILIKE '%data scientist%' OR
                            name ILIKE '%data science%' OR
                            name ILIKE '%исследователь данных%' OR
                            name LIKE '%ML%' OR
                            name ILIKE '%machine learning%' OR
                            name ILIKE '%машинн%обучен%'
                        ) AND
                            name NOT ILIKE'%HTML%'
                      ) AND
                    (
                        name ILIKE '%junior%' OR
                        experience ILIKE '%Нет опыта%' OR
                        employment ILIKE '%Стажировка%'
                    )
'''

In [39]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_6_2, conn)

df

,cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [40]:
query_6_3 = f'''SELECT COUNT(id) AS cnt
                FROM vacancies
                WHERE (
                        (
                            name ILIKE '%data scientist%' OR
                            name ILIKE '%data science%' OR
                            name ILIKE '%исследователь данных%' OR
                            name LIKE '%ML%' OR
                            name ILIKE '%machine learning%' OR
                            name ILIKE '%машинн%обучен%'
                        ) AND
                            name NOT ILIKE'%HTML%'
                      ) AND
                    (
                        key_skills ILIKE '%sql%' OR
                        key_skills ILIKE '%postres%'
                    )
'''

In [41]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_6_3, conn)

df

,cnt
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [42]:
query_6_4 = f'''SELECT COUNT(id) AS cnt
                FROM vacancies
                WHERE (
                        (
                            name ILIKE '%data scientist%' OR
                            name ILIKE '%data science%' OR
                            name ILIKE '%исследователь данных%' OR
                            name LIKE '%ML%' OR
                            name ILIKE '%machine learning%' OR
                            name ILIKE '%машинн%обучен%'
                        ) AND
                            name NOT ILIKE'%HTML%'
                      ) AND
                    (
                        key_skills ILIKE '%python%'
                    )
'''

In [43]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_6_4, conn)

df

,cnt
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [44]:
query_6_5 = f'''WITH cte AS (SELECT key_skills
                             FROM vacancies
                             WHERE (
                                     (
                                         name ILIKE '%data scientist%' OR
                                         name ILIKE '%data science%' OR
                                         name ILIKE '%исследователь данных%' OR
                                         name LIKE '%ML%' OR
                                         name ILIKE '%machine learning%' OR
                                         name ILIKE '%машинн%обучен%'
                                     ) AND
                                         name NOT ILIKE'%HTML%'
                                   )
                            )
                SELECT AVG(
                    LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1
                    ) AS avg_skills
                FROM cte
                WHERE key_skills IS NOT NULL
'''

In [45]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_6_5, conn)

df

,avg_skills
0,6.406032


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [46]:
query_6_6 = f'''WITH cte AS (SELECT experience,
                                    CASE
                                        WHEN COALESCE(salary_to, NULL) IS NOT NULL AND COALESCE(salary_from, NULL) IS NOT NULL THEN (salary_from + salary_to)/2
                                        WHEN COALESCE(NULL, salary_from) IS NOT NULL THEN salary_from
                                        WHEN COALESCE(NULL, salary_to) IS NOT NULL THEN salary_to
                                        ELSE NULL
                                    END AS salary
                             FROM vacancies
                             WHERE (
                                     (
                                         name ILIKE '%data scientist%' OR
                                         name ILIKE '%data science%' OR
                                         name ILIKE '%исследователь данных%' OR
                                         name LIKE '%ML%' OR
                                         name ILIKE '%machine learning%' OR
                                         name ILIKE '%машинн%обучен%'
                                     ) AND
                                         name NOT ILIKE'%HTML%'
                                   )
                            )

                SELECT experience,
                       ROUND(AVG(salary)) AS avg_salary
                FROM cte
                WHERE salary IS NOT NULL
                GROUP BY experience

'''

In [47]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_6_6, conn)

df

,experience,avg_salary
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


***

## Выводы:
* Предложений для начинающих DS на рынке не очень много.
* DS нужны навыки по анализу и сбору данных с помощью SQL.
* Кроме SQL среднем DS необходимо еще как минимум в среднем 6 навыков.

# Общий вывод по проекту

Поробуем выяснить какаие еще навыки у DS пользуются популярностью.

In [69]:
# К сожалению запрос, который выведет сразу такую сложную формулировку написать не получилось,
# но давайте запросим данные и дообработаем их с помощью других средств.
query_7 = f'''
    SELECT regexp_split_to_array(LOWER(key_skills),CHR(9)) AS ks
    FROM vacancies
    WHERE (
            (
                name ILIKE '%data scientist%' OR
                name ILIKE '%data science%' OR
                name ILIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                name ILIKE '%machine learning%' OR
                name ILIKE '%машинн%обучен%'
            ) AND
                name NOT ILIKE'%HTML%'
          ) AND key_skills IS NOT NULL
    
'''

In [70]:
with psycopg2.connect(**pass_data) as conn:
    df = pd.read_sql_query(query_7, conn)

df

,ks
0,"[python, sql, работа в команде, opencv, ml]"
1,"[oracle pl/sql, базы данных, machine learning,..."
2,"[python, opencv, tensorflow, computer vision, ..."
3,"[python, flask, linux, aws, git]"
4,"[python, sql, nlp, scikit-learn, machine learn..."
...,...
426,"[python, numpy, pytorch, pandas, математически..."
427,"[python, scikit-learn, pandas, git, numpy]"
428,"[qt, ооп, c++, ms visual studio, разработка ко..."
429,"[scala, sql, математический анализ, статистиче..."


In [84]:
from collections import Counter

In [85]:
# Дообработаем данные
skills = []
for i in df['ks'].to_list():
    skills+=i
    
skills = Counter(skills)

In [88]:
skills.most_common(5)

[('python', 348),
 ('sql', 191),
 ('machine learning', 140),
 ('git', 66),
 ('data science', 64)]

## Выводы:
* Получен навыки обращения с SQL зязыком запосов для PostresSQL. Все диалекты для MySQL, MSSQL, Oracle бладают своими особенностями, описанными в документации. Необходимо заметить, что Postre имеет обширную документаци на русском языке, что облегачает использование функционала даилекта.
* Наиболее востребованными навыками для DS по представленным вакансиям является **Python, SQL, ML, Git, Data Scince**.